# Webscrapeo : Ubica tu casilla

### Paquetes

In [126]:
import time
import requests
import numpy as np
import pandas as pd

### Pipeline

In [60]:
API = "https://us-central1-ine-ubica-tu-casilla.cloudfunctions.net/utc-api/search/rasgo-with-cmr-uts?e=1&s=488"
headers = {"authority": "us-central1-ine-ubica-tu-casilla.cloudfunctions.net",
            "method": "GET",
            "scheme": "https",
            "accept": "application/json, text/plain, */*",
            "accept-encoding": "gzip, deflate, br",
            "origin": "https://ubicatucasilla.ine.mx",
            "referer": "https://ubicatucasilla.ine.mx/",
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36",
            "x-utc-service-id": "ubicatucasilla",
            "x-utc-version-id": "2022"}

In [143]:
def get_raw_df(API, num_edo, num_casilla):
    """
    Extrae el dataframe de una casilla a partir del API
     
    Parameters
    API: string
        URL del API
    num_edo: int
        Numero de estado
    num_casilla: int
        Numero de casilla a extraer
    """
    path = "/utc-api/search/rasgo-with-cmr-uts?e={num_edo}&s={num_casilla}"
    headers["path"] = path 

    new_API = API.replace("e=1", f"e={num_edo}")
    new_API = new_API.replace("s=488", f"s={num_casilla}")
    r = requests.get(new_API, headers=headers)

    if r.json()['status'] == False:
        raw_df = pd.DataFrame([])
        content = {}
    else:
        content = r.json()['data'][0]['items']
        
        raw_df = pd.DataFrame(content[0]['apellidos'])
        raw_df['domicilio'] = content[0]['data']['domicilio']
        raw_df['ubicacion'] = content[0]['data']['ubicacion']

        raw_df['entidad'] = content[0]['entidadSede']
        raw_df['distitoSede'] = content[0]['distritoSede']
        raw_df['distritoLocal'] = content[0]['distrito_localSede']
        
        raw_df 

    return raw_df, content

In [144]:
def get_clean_df(raw_df, content):
    """
    Saca clean_df a partir del raw_df y pone varios 
    dfs de acuerdo al numero de secciones
    
    Parameters
    ----------
    raw_df: dataframe
        Dataframe de la casilla
    content: dict
        Diccionario con la informacion de la casilla
    
    """
    raw_secciones = content[0]['secciones']

    clean_secciones = [int(x) for x in raw_secciones]
    complete_df = pd.DataFrame([])

    for clean_seccion in clean_secciones:
        copy_df = raw_df.copy()
        copy_df['seccion'] = clean_seccion

        complete_df = pd.concat([complete_df, copy_df], 
                                ignore_index=True)

    return complete_df, clean_secciones

### Pipeline

In [160]:
all_secciones = [x for x in range(1, 7000)]
extracted_sections = []
estado_df = pd.DataFrame([])
counter = 0
for seccion in all_secciones:
    counter += 1
    if counter % 10 == 0:
        time.sleep(5)
    # La seccion ya la extrajimos
    if seccion in extracted_sections:
        continue

    raw_df, content = get_raw_df(API, 1, seccion)

    if content:
        # Sacar datos limpios y quitar casillas que ya hayamos extraido
        complete_df, clean_secciones = get_clean_df(raw_df, content)
        estado_df = pd.concat([estado_df, complete_df], ignore_index=True)
        # Agregar las secciones ya extraidas
        extracted_sections.extend(clean_secciones)

    # Llegamos al limite de secciones
    else:
        print(content)
        print("Limit reached, stopping...")
        print("secciones extraidas: ", seccion)
        break
        # no_existentes = [x for x in range(seccion, 7000)]
        # all_secciones = list(set(all_secciones) - set(no_existentes))

In [158]:
extracted_sections

[82,
 83,
 84,
 85,
 86,
 87,
 88,
 94,
 89,
 93,
 95,
 102,
 103,
 104,
 105,
 90,
 91,
 92,
 96,
 97,
 100,
 101]

In [159]:
complete_df

,casilla,inicial,final,domicilio,ubicacion,entidad,distitoSede,distritoLocal,seccion
0,B,ABAD GALVAN,ESPARZA MARTINEZ,"CALLE ALFA, NÚMERO 201, FRACCIONAMIENTO LA EST...",JARDÍN DE NIÑOS MARÍA CONCEPCIÓN MALDONADO,1,2,13,96
1,C1,ESPARZA MARTINEZ,LUCAS DOMINGUEZ,"CALLE ALFA, NÚMERO 201, FRACCIONAMIENTO LA EST...",JARDÍN DE NIÑOS MARÍA CONCEPCIÓN MALDONADO,1,2,13,96
2,C2,LUCERO HERRERA,RAMIREZ HERNANDEZ,"CALLE ALFA, NÚMERO 201, FRACCIONAMIENTO LA EST...",JARDÍN DE NIÑOS MARÍA CONCEPCIÓN MALDONADO,1,2,13,96
3,C3,RAMIREZ HERNANDEZ,ZUÑIGA ROSALES,"CALLE ALFA, NÚMERO 201, FRACCIONAMIENTO LA EST...",JARDÍN DE NIÑOS MARÍA CONCEPCIÓN MALDONADO,1,2,13,96
4,B,ABAD GALVAN,ESPARZA MARTINEZ,"CALLE ALFA, NÚMERO 201, FRACCIONAMIENTO LA EST...",JARDÍN DE NIÑOS MARÍA CONCEPCIÓN MALDONADO,1,2,13,97
5,C1,ESPARZA MARTINEZ,LUCAS DOMINGUEZ,"CALLE ALFA, NÚMERO 201, FRACCIONAMIENTO LA EST...",JARDÍN DE NIÑOS MARÍA CONCEPCIÓN MALDONADO,1,2,13,97
6,C2,LUCERO HERRERA,RAMIREZ HERNANDEZ,"CALLE ALFA, NÚMERO 201, FRACCIONAMIENTO LA EST...",JARDÍN DE NIÑOS MARÍA CONCEPCIÓN MALDONADO,1,2,13,97
7,C3,RAMIREZ HERNANDEZ,ZUÑIGA ROSALES,"CALLE ALFA, NÚMERO 201, FRACCIONAMIENTO LA EST...",JARDÍN DE NIÑOS MARÍA CONCEPCIÓN MALDONADO,1,2,13,97
8,B,ABAD GALVAN,ESPARZA MARTINEZ,"CALLE ALFA, NÚMERO 201, FRACCIONAMIENTO LA EST...",JARDÍN DE NIÑOS MARÍA CONCEPCIÓN MALDONADO,1,2,13,100
9,C1,ESPARZA MARTINEZ,LUCAS DOMINGUEZ,"CALLE ALFA, NÚMERO 201, FRACCIONAMIENTO LA EST...",JARDÍN DE NIÑOS MARÍA CONCEPCIÓN MALDONADO,1,2,13,100


In [130]:
all_secciones

[81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 175,
 176,
 177,
 178,
 179,
 180,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,

In [88]:
clean_secciones

[430, 448, 451, 488]

In [90]:
all_secciones = [x for x in range(1, 7000)]

In [91]:
pendientes = list(set(all_secciones) - set(clean_secciones))

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [147]:
raw_df, content = get_raw_df(API, 1, 2)

In [148]:
if content:
    print("Existe")

else:
    print("No existe")




Existe
